In [1]:
import pandas as pd
from binance import Client
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
coins = pd.read_excel("../database/export_binance/miguel_merged.xlsx")
coins = coins['Coin'].unique()
coins

array(['EUR', 'BTC', 'ETH', 'BNB', 'BUSD', 'ONG', 'ONT', 'USDT', 'AXS',
       'SLP', 'SOL', 'LUNA', 'MATIC', 'ADA', 'UST', 'ROSE', 'JASMY'],
      dtype=object)

In [4]:
len(coins)

17

In [6]:
api_key = os.getenv("BNB_API_KEY")
api_secret = os.getenv("BNB_SECRET_KEY")
client = Client(api_key, api_secret)
prices = pd.DataFrame()
for i in coins:
    if i == "USDT":
        pass
    elif i == "WETH":
        try:
            ticker = 'ETHUSDT'
            klines = pd.DataFrame(client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, "1 Jan, 2021", "10 Jun, 2022"), columns=["open_time", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
            klines['open_time'] = pd.to_datetime(klines.open_time, unit='ms')
            klines = klines.drop(['open', 'high', 'low', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore', 'close_time'], axis=1)
            klines.rename(columns={'open_time': 'datetime', 'close': 'WETHUSDT'}, inplace=True)
            klines.set_index('datetime', inplace=True)
            prices = pd.merge(prices, klines, how='outer', left_index=True, right_index=True)
        except:
            pass
    else:
        try:
            ticker = i + "USDT"
            klines = pd.DataFrame(client.get_historical_klines(ticker, Client.KLINE_INTERVAL_1DAY, "1 Jan, 2021", "10 Jun, 2022"), columns=["open_time", "open", "high", "low", "close", "volume", "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"])
            klines['open_time'] = pd.to_datetime(klines.open_time, unit='ms')
            klines = klines.drop(['open', 'high', 'low', 'volume', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore', 'close_time'], axis=1)
            klines.rename(columns={'open_time': 'datetime', 'close': ticker}, inplace=True)
            klines.set_index('datetime', inplace=True)
            prices = pd.merge(prices, klines, how='outer', left_index=True, right_index=True)
        except:
            klines = pd.DataFrame({ticker: []})
            prices = pd.merge(prices, klines, how='outer', left_index=True, right_index=True)
prices = prices.fillna(0)
prices

,EURUSDT,BTCUSDT,ETHUSDT,BNBUSDT,BUSDUSDT,ONGUSDT,ONTUSDT,AXSUSDT,SLPUSDT,SOLUSDT,LUNAUSDT,MATICUSDT,ADAUSDT,USTUSDT,ROSEUSDT,JASMYUSDT
datetime,,,,,,,,,,,,,,,,
2021-01-01,1.21830000,29331.69000000,728.91000000,37.77620000,0.99730000,0.19330000,0.44910000,0.59310000,0,1.84210000,0.64800000,0.01777000,0.17509000,0,0.04210000,0
2021-01-02,1.24120000,32178.33000000,774.56000000,38.23310000,0.99890000,0.20490000,0.46530000,0.54275000,0,1.79990000,0.62990000,0.01904000,0.17742000,0,0.04095000,0
2021-01-03,1.25940000,33000.05000000,978.28000000,41.25750000,0.99790000,0.20250000,0.50300000,0.56523000,0,2.17790000,0.65830000,0.01997000,0.20615000,0,0.04129000,0
2021-01-04,1.22860000,31988.71000000,1041.43000000,41.13330000,0.99930000,0.19080000,0.49450000,0.56082000,0,2.49090000,0.66720000,0.02468000,0.22528000,0,0.04135000,0
2021-01-05,1.22740000,33949.53000000,1099.56000000,41.82190000,0.99690000,0.19810000,0.50900000,0.55605000,0,2.16360000,0.71430000,0.02440000,0.25873000,0,0.04394000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-06,1.06800000,31373.10000000,1859.84000000,295.10000000,1.00090000,0.53210000,0.31320000,20.70000000,0.00533000,42.39000000,4.92160000,0.63300000,0.60910000,0,0.06825000,0.01213400
2022-06-07,1.07000000,31125.33000000,1813.33000000,290.30000000,1.00070000,0.47640000,0.30330000,19.72000000,0.00516000,39.31000000,3.82770000,0.61700000,0.61410000,0,0.06954000,0.01223000
2022-06-08,1.07200000,30204.77000000,1791.88000000,288.50000000,1.00080000,0.46110000,0.29690000,19.47000000,0.00504000,38.83000000,2.16910000,0.60600000,0.64080000,0,0.06600000,0.01183800


In [7]:
prices.to_excel('../database/prices/historical_prices_miguel.xlsx')